### SQL Superscript

#### Table of content

    1 Spark SQL
    1.1 Load modules/packages
    1.2 Spark connection setup
    1.3 Spark Sql selecting/transforming
    1.4 Complex Data-types transformation
    1.5 User defined functions
    1.6 Spark Sql Basic queries
    1.7 Spark Sql Advanced 1 queries
    

#### 1.1 Load modules/packages

In [3]:
from pyspark import SparkContext
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pyspark.sql import SparkSession 
from pyspark.sql import DataFrame 
from pyspark.sql import Column 
from pyspark.sql import Row 
from pyspark.sql import GroupedData 
from pyspark.sql import DataFrameNaFunctions       
from pyspark.sql import DataFrameStatFunctions    
from pyspark.sql import functions 
from pyspark.sql import types
from pyspark.sql import Window 
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import LongType
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.functions import expr
from pyspark.sql.functions import from_unixtime

#### 1.2 Spark connection setup

In [4]:
from pyspark import SparkContext
# Create the sc
sc = SparkContext.getOrCreate()

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#### 1.3 Spark SQL selecting/transforming

In [5]:
# Read Json file
df = spark.read.json("C:/big-datademo/superscripts/Python/data/project.json")
# Display thethe DataFrame 
df.show()

+------+-------+
|profit|project|
+------+-------+
|  null|      A|
| 12000|      B|
| 24000|      C|
| 36000|      D|
| 48000|      E|
+------+-------+



In [6]:
# Printing schema in tree format
df.printSchema()

root
 |-- profit: long (nullable = true)
 |-- project: string (nullable = true)



In [7]:
# Selecting "name" column
df.select("project").show()

+-------+
|project|
+-------+
|      A|
|      B|
|      C|
|      D|
|      E|
+-------+



In [8]:
# Selecting everybody, but incrementing profit by 1
df.select(df['project'], df['profit'] + 1).show()

+-------+------------+
|project|(profit + 1)|
+-------+------------+
|      A|        null|
|      B|       12001|
|      C|       24001|
|      D|       36001|
|      E|       48001|
+-------+------------+



In [9]:
# Selecting projects with profit higher than 40000
df.filter(df['profit'] > 40000).show()

+------+-------+
|profit|project|
+------+-------+
| 48000|      E|
+------+-------+



In [10]:
# Count projects by profit
df.groupBy("profit").count().show()

+------+-----+
|profit|count|
+------+-----+
|  null|    1|
| 48000|    1|
| 36000|    1|
| 24000|    1|
| 12000|    1|
+------+-----+



In [11]:
# Registering the DataFrame as a SQL temporary view
df.createOrReplaceTempView("project")

In [12]:
sqlDF = spark.sql("SELECT * FROM project")
sqlDF.show()

+------+-------+
|profit|project|
+------+-------+
|  null|      A|
| 12000|      B|
| 24000|      C|
| 36000|      D|
| 48000|      E|
+------+-------+



In [13]:
# Registering the DataFrame as a global temporary view
df.createGlobalTempView("project")

In [14]:
# Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.project").show()

+------+-------+
|profit|project|
+------+-------+
|  null|      A|
| 12000|      B|
| 24000|      C|
| 36000|      D|
| 48000|      E|
+------+-------+



In [15]:
# Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.project").show()

+------+-------+
|profit|project|
+------+-------+
|  null|      A|
| 12000|      B|
| 24000|      C|
| 36000|      D|
| 48000|      E|
+------+-------+



In [16]:
from pyspark.sql import Row

sc = spark.sparkContext

# Loading a text file and convert each line to a Row.
lines = sc.textFile("C:/big-datademo/superscripts/Python/data/project.txt")
parts = lines.map(lambda l: l.split(","))
project = parts.map(lambda p: Row(project=p[0], profit=int(p[1])))

# Infer the schema, and register the DataFrame as a table.
schemaPeople = spark.createDataFrame(project)
schemaPeople.createOrReplaceTempView("project")

# SQL runned over DataFrames that have been registered as a table.
highprofit = spark.sql("SELECT project FROM project WHERE profit >= 40000 AND profit <= 100000")

# The results of SQL queries are Dataframe objects. Rdd returning the content as an :class:`pyspark.RDD` of :class:`Row`.
projectName = highprofit.rdd.map(lambda p: "project: " + p.project).collect()
for project in projectName:
    print(project)

project: D
project: E


In [17]:
# Import data types
from pyspark.sql.types import *

sc = spark.sparkContext

# Loading a text file and converting each line to a Row.
lines = sc.textFile("C:/big-datademo/superscripts/Python/data/project.txt")
parts = lines.map(lambda l: l.split(","))

# Each line is converted to a tuple.
project = parts.map(lambda p: (p[0], p[1].strip()))

# The schema is encoded in a string.
schemaString = "project profit"

fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Applying the schema to the RDD.
schemaProject = spark.createDataFrame(project, schema)

# Creating a temporary view using the DataFrame
schemaProject.createOrReplaceTempView("project")

# SQL runned over DataFrames that have been registered as a table.
results = spark.sql("SELECT project FROM project")

results.show()

+-------+
|project|
+-------+
|      A|
|      B|
|      C|
|      D|
|      E|
+-------+



In [19]:
df = spark.read.load("C:/big-datademo/superscripts/Python/data/users.parquet")
df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")

In [41]:
df = spark.read.load("C:/big-datademo/superscripts/Python/data/project.json", format="json")
df.select("project", "profit").write.save("ProjectProfit.parquet", format="parquet")

In [20]:
df = spark.read.load("C:/big-datademo/superscripts/Python/data/project.csv",
                     format="csv", sep=":", inferSchema="true", header="true")

In [21]:
df = spark.sql("SELECT * FROM parquet.`C:/big-datademo/superscripts/Python/data/users.parquet`")

In [ ]:
df.write.bucketBy(42, "name").sortBy("favorite_numbers").saveAsTable("people_bucketed2")

In [52]:
df.write.partitionBy("favorite_color").format("parquet").save("namesPartByColor.parquet")

In [ ]:
df = spark.read.parquet("C:/big-datademo/superscripts/Python/data/users.parquet")
(df
    .write
    .partitionBy("favorite_color")
    .bucketBy(42, "name")
    .saveAsTable("people_partitioned_bucketed1"))

In [25]:
projectDF = spark.read.json("C:/big-datademo/superscripts/Python/data/project.json")

# DataFrames can be saved as Parquet files, maintaining the schema information.
projectDF.write.parquet("project.parquet")

# Reading in the Parquet file created above. The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("project.parquet")

# Parquet files used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
highprofit = spark.sql("SELECT project FROM parquetFile WHERE profit >= 40000 AND profit <= 100000")
highprofit.show()

+-------+
|project|
+-------+
|      E|
+-------+



In [ ]:
from pyspark.sql import Row

# Creating a simple DataFrame, stored into a partition directory
sc = spark.sparkContext

squaresDF = spark.createDataFrame(sc.parallelize(range(1, 6))
                                  .map(lambda i: Row(single=i, double=i ** 2)))
squaresDF.write.parquet("data/test_table/key=1")

# Creating another DataFrame in a new partition directory, adding a new column and dropping an existing column
cubesDF = spark.createDataFrame(sc.parallelize(range(6, 11))
                                .map(lambda i: Row(single=i, triple=i ** 3)))
cubesDF.write.parquet("data/test_table/key=2")

# Reading the partitioned table
mergedDF = spark.read.option("mergeSchema", "true").parquet("data/test_table")
mergedDF.printSchema()

In [65]:

sc = spark.sparkContext

# The JSON dataset is pointed to by path. 
path = "C:/big-datademo/superscripts/Python/data/project.json"
projectDF = spark.read.json(path)

# The inferred schema visualized using printSchema() 
projectDF.printSchema()

# Creating a temporary view using the DataFrame
projectDF.createOrReplaceTempView("project")

# SQL statement
projectNamesDF = spark.sql("SELECT project FROM project WHERE profit BETWEEN 20000 AND 40000")


root
 |-- profit: long (nullable = true)
 |-- project: string (nullable = true)

+-------+
|project|
+-------+
|      C|
|      D|
+-------+



In [66]:
# Alternatively, a DataFrame is created for a JSON dataset represented by an RDD[String] storing one JSON object per string
jsonStrings = ['{"project":"C","Team":{"city":"New York City","State":"New york"}}']
otherProjectRDD = sc.parallelize(jsonStrings)
otherProject = spark.read.json(otherProjectRDD)
otherProject.show()

+--------------------+-------+
|                Team|project|
+--------------------+-------+
|[New york, New Yo...|      C|
+--------------------+-------+



In [3]:
from os.path import expanduser, join, abspath
from pyspark.sql import SparkSession
from pyspark.sql import Row

# The arehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

# spark as an existing SparkSession
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("LOAD DATA LOCAL INPATH 'C:/big-datademo/superscripts/Python/data/kv1.txt' INTO TABLE src")

# Queries expressed in HiveQL
spark.sql("SELECT * FROM src").show()

# Aggregation queries 
spark.sql("SELECT COUNT(*) FROM src").show()

# The results of SQL queries; DataFrames supporting all normal functions.
sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 10 ORDER BY key")

# The items in DataFrames are of type Row, allowing access to each column by ordinal.
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)
    
# Using DataFrames to create temporary views within a SparkSession.
Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

# joining DataFrame data with with queries and data stored in Hive.
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
| 86| val_86|
|311|val_311|
| 27| val_27|
|165|val_165|
|409|val_409|
|255|val_255|
|278|val_278|
| 98| val_98|
|484|val_484|
|265|val_265|
|193|val_193|
|401|val_401|
|150|val_150|
|273|val_273|
|224|val_224|
|369|val_369|
| 66| val_66|
|128|val_128|
|213|val_213|
+---+-------+
only showing top 20 rows

+--------+
|count(1)|
+--------+
|    4500|
+--------+

Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: None
Key: 0, Value: None
Key: 0, Value: None
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 0, Value: val_0
Key: 2, Value: val

In [27]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

# Declaring the function and creating the UDF
def multiply_func(a, b):
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
print(multiply_func(x, x))

# Creating a Spark DataFrame, 'spark' is an existing SparkSession
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

df.show()

0    1
1    4
2    9
dtype: int64
+---+
|  x|
+---+
|  1|
|  2|
|  3|
+---+



In [47]:
import numpy as np
import pandas as pd

# Enabling Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# Generating a Pandas DataFrame
pdf = pd.DataFrame(np.random.rand(100, 3))

# Creating a Spark DataFrame from a Pandas DataFrame using Arrow
df = spark.createDataFrame(pdf)

# Converting the Spark DataFrame back to a Pandas DataFrame using Arrow
result_pdf = df.select("*").toPandas()

#### 1.4 Complex Data-types transformation 

In [9]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Function for turning JSON strings into DataFrames.
def jsonToDataFrame(json, schema=None):
  # SparkSessions are available with Spark 2.0+
  reader = spark.read
  if schema:
    reader.schema(schema)
  return reader.json(sc.parallelize([json]))

In [37]:
# Using a struct
schema = StructType().add("a", StructType().add("b", IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

display(events.select("a.b"))

events.show()

DataFrame[b: int]

+---+
|  a|
+---+
|[1]|
+---+



In [36]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))
                          
events = jsonToDataFrame("""
{
  "a": {
     "b": 1
  }
}
""", schema)

display(events.select("a.b"))

events.show()

DataFrame[b: int]

+--------+
|       a|
+--------+
|[b -> 1]|
+--------+



In [35]:
events = jsonToDataFrame("""
{
  "a": {
     "b": 1,
     "c": 2
  }
}
""")

display(events.select("a.*"))

events.show()

DataFrame[b: bigint, c: bigint]

+------+
|     a|
+------+
|[1, 2]|
+------+



In [34]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2,
  "c": 3
}
""")

display(events.select(struct(col("a").alias("y")).alias("x")))

events.show()

DataFrame[x: struct<y:bigint>]

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
+---+---+---+



In [33]:
events = jsonToDataFrame("""
{
  "a": 1,
  "b": 2
}
""")

display(events.select(struct("*").alias("x")))

events.show()

DataFrame[x: struct<a:bigint,b:bigint>]

+---+---+
|  a|  b|
+---+---+
|  1|  2|
+---+---+



In [32]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

display(events.select(col("a").getItem(0).alias("x")))

events.show()


DataFrame[x: bigint]

+------+
|     a|
+------+
|[1, 2]|
+------+



In [31]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1
  }
}
""", schema)

display(events.select(col("a").getItem("b").alias("x")))

events.show()

DataFrame[x: int]

+--------+
|       a|
+--------+
|[b -> 1]|
+--------+



In [30]:
events = jsonToDataFrame("""
{
  "a": [1, 2]
}
""")

display(events.select(explode("a").alias("x")))

events.show()

DataFrame[x: bigint]

+------+
|     a|
+------+
|[1, 2]|
+------+



In [29]:
# Using a map
schema = StructType().add("a", MapType(StringType(), IntegerType()))

events = jsonToDataFrame("""
{
  "a": {
    "b": 1,
    "c": 2
  }
}
""", schema)

display(events.select(explode("a").alias("x", "y")))

events.show()

DataFrame[x: string, y: int]

+----------------+
|               a|
+----------------+
|[b -> 1, c -> 2]|
+----------------+



In [26]:
events = jsonToDataFrame("""
[{ "x": 1 }, { "x": 2 }]
""")

display(events.select(collect_list("x").alias("x")))

events.show()

DataFrame[x: array<bigint>]

+---+
|  x|
+---+
|  1|
|  2|
+---+



In [25]:
# using an aggregation
events = jsonToDataFrame("""
[{ "x": 1, "y": "a" }, { "x": 2, "y": "b" }]
""")

display(events.groupBy("y").agg(collect_list("x").alias("x")))

events.show()

DataFrame[y: string, x: array<bigint>]

+---+---+
|  x|  y|
+---+---+
|  1|  a|
|  2|  b|
+---+---+



In [38]:
events = jsonToDataFrame("""
{
  "a": [
    {"b": 1},
    {"b": 2}
  ]
}
""")

display(events.select("a.b"))



DataFrame[b: array<bigint>]

In [39]:
events = jsonToDataFrame("""
{
  "a": {
    "b": 1
  }
}
""")

display(events.select(to_json("a").alias("c")))

DataFrame[c: string]

In [40]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":1}"
}
""")

schema = StructType().add("b", IntegerType())
display(events.select(from_json("a", schema).alias("c")))

DataFrame[c: struct<b:int>]

In [41]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":{\\"x\\":1,\\"y\\":{\\"z\\":2}}}"
}
""")

schema = StructType().add("b", StructType().add("x", IntegerType())
                            .add("y", StringType()))
display(events.select(from_json("a", schema).alias("c")))

DataFrame[c: struct<b:struct<x:int,y:string>>]

In [42]:
events = jsonToDataFrame("""
{
  "a": "{\\"b\\":1}"
}
""")

display(events.select(json_tuple("a", "b").alias("c")))


DataFrame[c: string]

In [43]:
events = jsonToDataFrame("""
[{ "a": "x: 1" }, { "a": "y: 2" }]
""")

display(events.select(regexp_extract("a", "([a-z]):", 1).alias("c")))



DataFrame[c: string]

#### 1.5 User defined functions

In [74]:
def squared(s):
  return s * s
sqlContext.udf.register("squaredWithPython", squared)

<function __main__.squared(s)>

In [75]:
from pyspark.sql.types import LongType
def squared_typed(s):
  return s * s
sqlContext.udf.register("squaredWithPython", squared, LongType())

<function __main__.squared(s)>

In [82]:
sqlContext.range(1, 20).registerTempTable("test")

In [108]:
df = spark.sql("select * FROM test") 

df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [107]:
df = spark.sql("select squaredWithPython(id) as id_squared FROM test") 

df.show()

+----------+
|id_squared|
+----------+
|         1|
|         4|
|         9|
|        16|
|        25|
|        36|
|        49|
|        64|
|        81|
|       100|
|       121|
|       144|
|       169|
|       196|
|       225|
|       256|
|       289|
|       324|
|       361|
+----------+



In [114]:
from pyspark.sql.functions import udf
squared_udf = udf(squared, LongType())
df = sqlContext.table("test")
display(df.select("id", squared_udf("id").alias("id_squared")))

DataFrame[id: bigint, id_squared: bigint]

#### 1.6 Spark Sql basic queries

In [173]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import from_unixtime

df = spark.read.csv("C:/big-datademo/superscripts/Python/databricks/export.csv",
                     header="true")

df.show()


+------+----------+
|action|      date|
+------+----------+
| Close|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
| Close|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
+------+----------+
only showing top 20 rows



In [181]:
# Registering the DataFrame as a SQL temporary view
df.createOrReplaceTempView("action")

df = spark.sql("select * FROM action") 

df.show()

+------+----------+
|action|      date|
+------+----------+
| Close|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
| Close|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
|  Open|28/07/2016|
| Close|28/07/2016|
+------+----------+
only showing top 20 rows



In [184]:
df = spark.sql("SELECT count(*) FROM action") 

df.show()

+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [192]:
df = spark.sql("SELECT count(*) FROM action") 
df.show()           

+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [197]:
df = spark.sql("
               
select date, action, count(action) as action_count from action 
group by action, date 
order by date, action desc") 
               
df.show() 

+----------+------+------------+
|      date|action|action_count|
+----------+------+------------+
|28/07/2016|  Open|         498|
|28/07/2016| Close|         502|
+----------+------+------------+



In [2]:
df = spark.read.csv("C:/big-datademo/superscripts/Python/databricks/Airlines/2008.csv",
                     header="true")

In [6]:
# Registering the DataFrame as a SQL temporary view
df.createOrReplaceTempView("flights")

df = spark.sql("select * FROM flights") 

df.head()

Row(Year='2008', Month='1', DayofMonth='3', DayOfWeek='4', DepTime='2003', CRSDepTime='1955', ArrTime='2211', CRSArrTime='2225', UniqueCarrier='WN', FlightNum='335', TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150', AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', Distance='810', TaxiIn='4', TaxiOut='8', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [5]:
df = spark.sql("""DROP TABLE IF EXISTS flights""")



In [15]:
df = spark.sql(("""CREATE TABLE flights
USING parquet
PARTITIONED BY (Origin)
SELECT _c0 as Year, _c1 as Month, _c2 as DayofMonth, _c3 as DayOfWeek, _c4 as DepartureTime, _c5 as CRSDepartureTime, _c6 as ArrivalTime, 
  _c7 as CRSArrivalTime, _c8 as UniqueCarrier, _c9 as FlightNumber, _c10 as TailNumber, _c11 as ActualElapsedTime, _c12 as CRSElapsedTime, 
    _c13 as AirTime, _c14 as ArrivalDelay, _c15 as DepartureDelay, _c16 as Origin, _c17 as Destination, _c18 as Distance, 
    _c19 as TaxiIn, _c20 as TaxiOut, _c21 as Cancelled, _c22 as CancellationCode, _c23 as Diverted, _c24 as CarrierDelay, 
    _c25 as WeatherDelay, _c26 as NASDelay, _c27 as SecurityDelay, _c28 as LateAircraftDelay 
FROM csv.`C:/big-datademo/superscripts/Python/databricks/Airlines/2008.csv`"""))



In [16]:
df = spark.sql("select * FROM flights") 
df.head()

Row(Year='2008', Month='3', DayofMonth='2', DayOfWeek='7', DepartureTime='1335', CRSDepartureTime='1155', ArrivalTime='1606', CRSArrivalTime='1419', UniqueCarrier='CO', FlightNumber='84', TailNumber='N16642', ActualElapsedTime='151', CRSElapsedTime='144', AirTime='122', ArrivalDelay='107', DepartureDelay='100', Destination='EWR', Distance='745', TaxiIn='8', TaxiOut='21', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='0', WeatherDelay='0', NASDelay='107', SecurityDelay='0', LateAircraftDelay='0', Origin='ATL')

In [36]:
# Selecting 
df = spark.sql("""

SELECT  Year AS Year, 
        Month AS Month, 
        DayofMonth AS Day,
        AirTime AS Airduration
FROM flights
LIMIT 20

""").show()

+----+-----+---+-----------+
|Year|Month|Day|Airduration|
+----+-----+---+-----------+
|2008|    3|  2|        122|
|2008|    5|  9|         81|
|2008|    3|  3|        104|
|2008|    5| 11|         93|
|2008|    3|  4|         94|
|2008|    5| 22|         84|
|2008|    3|  5|        111|
|2008|    5| 13|         82|
|2008|    3|  6|         93|
|2008|    5|  4|        110|
|2008|    3|  7|         90|
|2008|    5|  9|         81|
|2008|    3|  9|        101|
|2008|    5| 14|        108|
|2008|    3| 10|        125|
|2008|    5|  2|        102|
|2008|    3| 11|        108|
|2008|    5| 29|         81|
|2008|    3| 12|        119|
|2008|    5| 18|        109|
+----+-----+---+-----------+



In [24]:
# ORDER BY 
df = spark.sql("""

SELECT   
        Month AS Month, 
        DayofMonth AS Day,
        AirTime AS Airduration
FROM flights
ORDER BY Month ASC
LIMIT 20

""").show()

+-----+---+-----------+
|Month|Day|Airduration|
+-----+---+-----------+
|    1|  3|        116|
|    1|  3|        113|
|    1|  3|         76|
|    1|  3|         78|
|    1|  3|         77|
|    1|  3|         87|
|    1|  3|        230|
|    1|  3|        219|
|    1|  3|         70|
|    1|  3|         70|
|    1|  3|        106|
|    1|  3|        107|
|    1|  3|         39|
|    1|  3|         37|
|    1|  3|         35|
|    1|  3|         37|
|    1|  3|        213|
|    1|  3|        205|
|    1|  3|        110|
|    1|  3|         49|
+-----+---+-----------+



In [38]:
# WHERE 
df = spark.sql("""

SELECT  Month AS Month, 
        DayofMonth AS Day,
        AirTime AS Airduration
FROM flights
WHERE Airtime BETWEEN 0 AND 200
ORDER BY Month ASC
LIMIT 5

""").show()
        

+-----+---+-----------+
|Month|Day|Airduration|
+-----+---+-----------+
|    1|  3|        116|
|    1|  3|        113|
|    1|  3|         76|
|    1|  3|         78|
|    1|  3|         77|
+-----+---+-----------+



In [51]:
# LIKE 
df = spark.sql("""

SELECT  Month AS Month, 
        DayofMonth AS Day,
        AirTime AS Airduration,
        Dest AS Destination
FROM flights 
WHERE Dest LIKE 'T%' OR Dest LIKE '%I' AND Airtime BETWEEN 0 AND 300
ORDER BY Month ASC
LIMIT 10

""").show()

+-----+---+-----------+-----------+
|Month|Day|Airduration|Destination|
+-----+---+-----------+-----------+
|    1|  3|        116|        TPA|
|    1|  3|        113|        TPA|
|    1|  3|         76|        BWI|
|    1|  3|         78|        BWI|
|    1|  3|         77|        BWI|
|    1|  3|         70|        MCI|
|    1|  3|         70|        MCI|
|    1|  3|        110|        TPA|
|    1|  3|         49|        BWI|
|    1|  3|         51|        BWI|
+-----+---+-----------+-----------+



In [46]:
# Calculated columns
df = spark.sql("""
SELECT  Month, 
        DayofMonth,
        AirTime * 0.9 AS CorrectedAirtime,
        Dest,
        ArrDelay,
        AirTime+ArrDelay AS InAirTime
FROM flights 
WHERE AirTime > 100
LIMIT 5
""").show()

+-----+----------+----------------+----+--------+---------+
|Month|DayofMonth|CorrectedAirtime|Dest|ArrDelay|InAirTime|
+-----+----------+----------------+----+--------+---------+
|    1|         3|           104.4| TPA|     -14|    102.0|
|    1|         3|           101.7| TPA|       2|    115.0|
|    1|         3|           207.0| LAS|      57|    287.0|
|    1|         3|           197.1| LAS|     -18|    201.0|
|    1|         3|            95.4| MCO|       1|    107.0|
+-----+----------+----------------+----+--------+---------+



In [78]:
# CASE when
df = spark.sql("""
SELECT  Month, Airtime, 
CASE WHEN AirTime <= 50 THEN 'Short'
     WHEN AirTime <= 100 THEN 'Medium'
     WHEN AirTime <= 200 THEN 'long'
     ELSE 'Very Long' END
FROM flights 
LIMIT 5
""").show()

+-----+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Month|Airtime|CASE WHEN (CAST(AirTime AS INT) <= 50) THEN Short WHEN (CAST(AirTime AS INT) <= 100) THEN Medium WHEN (CAST(AirTime AS INT) <= 200) THEN long ELSE Very Long END|
+-----+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    1|    116|                                                                                                                                                            long|
|    1|    113|                                                                                                                                                            long|
|    1|     76|                                                                                                    

In [79]:
# CASE when LIKE 
df = spark.sql("""
SELECT  Month, Airtime, Dest,
CASE WHEN Dest LIKE 'A%' THEN 'Premium Flights'
     ELSE 'Budget Flights' END
FROM flights 
LIMIT 5
""").show()

+-----+-------+----+-------------------------------------------------------------------+
|Month|Airtime|Dest|CASE WHEN Dest LIKE A% THEN Premium Flights ELSE Budget Flights END|
+-----+-------+----+-------------------------------------------------------------------+
|    1|    116| TPA|                                                     Budget Flights|
|    1|    113| TPA|                                                     Budget Flights|
|    1|     76| BWI|                                                     Budget Flights|
|    1|     78| BWI|                                                     Budget Flights|
|    1|     77| BWI|                                                     Budget Flights|
+-----+-------+----+-------------------------------------------------------------------+



In [25]:
#Getting top 20 cities with highest monthly total flights on first day of week
df = spark.sql("""SELECT Month, Origin, count(*) as TotalFlights 
FROM flights
WHERE DayOfWeek = 1 
GROUP BY Month, Origin 
ORDER BY TotalFlights DESC
LIMIT 20""") 
               
df.show() 


+-----+------+------------+
|Month|Origin|TotalFlights|
+-----+------+------------+
|    6|   ATL|        6046|
|    3|   ATL|        6019|
|   12|   ATL|        5800|
|    9|   ATL|        5722|
|    6|   ORD|        5241|
|    3|   ORD|        5072|
|    9|   ORD|        4931|
|    7|   ATL|        4894|
|    8|   ATL|        4821|
|    4|   ATL|        4798|
|   11|   ATL|        4776|
|   10|   ATL|        4684|
|    5|   ATL|        4656|
|    2|   ATL|        4601|
|    1|   ATL|        4540|
|   12|   ORD|        4473|
|    7|   ORD|        4249|
|    8|   ORD|        4171|
|    4|   ORD|        4140|
|    5|   ORD|        4134|
+-----+------+------------+



In [87]:
df.head()

Row(Year='2008', Month='1', DayofMonth='3', DayOfWeek='4', DepTime='2003', CRSDepTime='1955', ArrTime='2211', CRSArrTime='2225', UniqueCarrier='WN', FlightNum='335', TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150', AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', Distance='810', TaxiIn='4', TaxiOut='8', Cancelled='0', CancellationCode=None, Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')

In [5]:
df = spark.read.csv("C:/big-datademo/superscripts/Python/databricks/Airlines/2008.csv",
                     header="true")

df.createOrReplaceTempView("airlines")

In [29]:
df = spark.sql("select Year,AirTime  from flights limit 10") 
df.show()

+----+-------+
|Year|AirTime|
+----+-------+
|2008|    122|
|2008|     81|
|2008|    104|
|2008|     93|
|2008|     94|
|2008|     84|
|2008|    111|
|2008|     82|
|2008|     93|
|2008|    110|
+----+-------+



In [103]:
df = spark.sql("select * from airlines where Airtime >1000") 
df.show()


+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    5|        24|        6|    920|       920|    519|      1205|           HA|       21

In [115]:
#Counting number of rows
df = spark.sql("""select count(*) as TPA_count from airlines where Dest='TPA'""") 
df.show()

+---------+
|TPA_count|
+---------+
|    78171|
+---------+



In [231]:
#SQL like
df = spark.sql("""select * from airlines where Dest like 'T%' Limit 3""") 
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [120]:
df = spark.sql("""select Year,Month,Dest from airlines """) 
df.show()

+----+-----+----+
|Year|Month|Dest|
+----+-----+----+
|2008|    1| TPA|
|2008|    1| TPA|
|2008|    1| BWI|
|2008|    1| BWI|
|2008|    1| BWI|
|2008|    1| JAX|
|2008|    1| LAS|
|2008|    1| LAS|
|2008|    1| MCI|
|2008|    1| MCI|
|2008|    1| MCO|
|2008|    1| MCO|
|2008|    1| MDW|
|2008|    1| MDW|
|2008|    1| MDW|
|2008|    1| MDW|
|2008|    1| PHX|
|2008|    1| PHX|
|2008|    1| TPA|
|2008|    1| BWI|
+----+-----+----+
only showing top 20 rows



In [232]:
#SQL where with and clause
df = spark.sql("""select * from airlines where Dest like 'JA%'and (Month = 6 or Month = 12) Limit 3""") 
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    6|         2|        1|   1138|      1140|   1224|      1230|           WN|      201

In [159]:
#SQL IN clause
df = spark.sql("""

select * from airlines 
where Month in (6,12)
LIMIT 3

""").show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    6|         2|        1|   1956|      1955|   2050|      2055|           WN|       34

In [31]:
# Group By

df = spark.sql("""

select Dest, count(*) as count
from airlines 
group by Dest
LIMIT 3

""").show()


+----+-----+
|Dest|count|
+----+-----+
| BGM|  728|
| PSE|  753|
| DLG|  116|
+----+-----+



In [32]:
# Group By with having clause

df = spark.sql(""" 

select Dest, count(*) as count
from airlines 
group by Dest having count > 5
LIMIT 3

""").show()


+----+-----+
|Dest|count|
+----+-----+
| BGM|  728|
| PSE|  753|
| DLG|  116|
+----+-----+



In [33]:
# SQL Order by

df = spark.sql("""

select Dest, count(*) as count
from airlines group by Dest having count > 5 order by Dest


""").show()

+----+------+
|Dest| count|
+----+------+
| ABE|  4795|
| ABI|  2661|
| ABQ| 41144|
| ABY|  1095|
| ACK|   469|
| ACT|  1994|
| ACV|  3702|
| ACY|   113|
| ADK|   102|
| ADQ|   706|
| AEX|  2327|
| AGS|  2406|
| AKN|   116|
| ALB| 13468|
| ALO|   323|
| AMA|  7490|
| ANC| 19329|
| ASE|  5243|
| ATL|414521|
| ATW|  5872|
+----+------+
only showing top 20 rows



In [8]:
promoted_content = spark.read.csv("C:/big-datademo/superscripts/data/Airline data/promoted_content.csv",
                     header="true")

train_clicks = spark.read.csv("C:/big-datademo/superscripts/data/Airline data/train_clicks.csv",
                     header="true")

events = spark.read.csv("C:/big-datademo/superscripts/data/Airline data/events.csv",
                     header="true")

promoted_content.createOrReplaceTempView("Table_promoted_content")
train_clicks.createOrReplaceTempView("Table_train_clicks")
events.createOrReplaceTempView("Table_events")

In [25]:
# Inner Join
df = spark.sql("""

select TPC.*,TTC.*
from Table_train_clicks TTC
inner join Table_promoted_content TPC
on TTC.ad_id = TPC.ad_id""").show()

+-----+-----------+-----------+-------------+----------+-----+-------+
|ad_id|document_id|campaign_id|advertiser_id|display_id|ad_id|clicked|
+-----+-----------+-----------+-------------+----------+-----+-------+
|   35|      29658|         24|           19|      8157|   35|      0|
|  216|      55604|         48|           39|      9646|  216|      1|
|  304|      56139|         53|           86|     11384|  304|      0|
|  333|      58767|         53|           86|      3738|  333|      0|
|  387|      60946|         59|           76|      9292|  387|      0|
|  405|      61172|         42|           76|     18092|  405|      0|
|  410|      61553|         53|           86|      3738|  410|      0|
|  591|      91275|         53|           86|      2957|  591|      0|
|  605|    1045711|         90|          138|     14402|  605|      0|
|  675|     133963|         53|           86|     10426|  675|      0|
|  917|     151471|        142|          241|      8994|  917|      0|
|  917

In [50]:
# Left Join
df = spark.sql("""

select TPC.*,TTC.*
from Table_train_clicks TTC
left join Table_promoted_content TPC
on TTC.ad_id = TPC.ad_id

""").show()

+-----+-----------+-----------+-------------+----------+------+-------+
|ad_id|document_id|campaign_id|advertiser_id|display_id| ad_id|clicked|
+-----+-----------+-----------+-------------+----------+------+-------+
|42337|     938164|       5969|         1499|         1| 42337|      0|
| null|       null|       null|         null|         1|139684|      0|
| null|       null|       null|         null|         1|144739|      1|
| null|       null|       null|         null|         1|156824|      0|
| null|       null|       null|         null|         1|279295|      0|
| null|       null|       null|         null|         1|296965|      0|
| null|       null|       null|         null|         2|125211|      0|
| null|       null|       null|         null|         2|156535|      0|
| null|       null|       null|         null|         2|169564|      0|
| null|       null|       null|         null|         2|308455|      1|
|71547|    1043039|       8711|         1919|         3| 71547| 

In [51]:
# Right Join
df = spark.sql("""

select TPC.*,TTC.*
from Table_train_clicks TTC
right join Table_promoted_content TPC
on TTC.ad_id = TPC.ad_id

""").show()

+-----+-----------+-----------+-------------+----------+-----+-------+
|ad_id|document_id|campaign_id|advertiser_id|display_id|ad_id|clicked|
+-----+-----------+-----------+-------------+----------+-----+-------+
|    1|       6614|          1|            7|      null| null|   null|
|    2|     471467|          2|            7|      null| null|   null|
|    3|       7692|          3|            7|      null| null|   null|
|    4|     471471|          2|            7|      null| null|   null|
|    5|     471472|          2|            7|      null| null|   null|
|    6|      12736|          1|            7|      null| null|   null|
|    7|      12808|          1|            7|      null| null|   null|
|    8|     471477|          2|            7|      null| null|   null|
|    9|      13379|          1|            7|      null| null|   null|
|   10|      13885|          1|            7|      null| null|   null|
|   11|      14230|          1|            7|      null| null|   null|
|   12

In [13]:
# Union
df = spark.sql("""

select clicked FROM Table_train_clicks WHERE ad_id = 144739
UNION ALL
select advertiser_id FROM Table_promoted_content WHERE ad_id = 144739
""").show()

+-------+
|clicked|
+-------+
|      1|
|      1|
|      0|
|      1|
|      1|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      1|
|      0|
|      0|
|      0|
|      1|
+-------+
only showing top 20 rows



In [34]:
# Distinct
df = spark.sql("""

select distinct Dest from airlines
LIMIT 3

""").show()

+----+
|Dest|
+----+
| BGM|
| PSE|
| DLG|
+----+



In [186]:
# Avg
df = spark.sql("""

select avg(Airtime) from airlines
LIMIT 100

""").show()

+----------------------------+
|avg(CAST(Airtime AS DOUBLE))|
+----------------------------+
|           104.0185891263188|
+----------------------------+



In [190]:
# Max
df = spark.sql("""

select max(Airtime) from airlines

""").show()

+------------+
|max(Airtime)|
+------------+
|          NA|
+------------+



In [192]:
# Mean
df = spark.sql("""

select mean(Airtime) from airlines

""").show()

+----------------------------+
|avg(CAST(Airtime AS DOUBLE))|
+----------------------------+
|           104.0185891263188|
+----------------------------+



In [ ]:
# Calculated columns
df = spark.sql("""
SELECT  Month, 
        DayofMonth,
        AirTime * 0.9 AS CorrectedAirtime,
        Dest,
        ArrDelay,
        AirTime+ArrDelay AS InAirTime
FROM flights 
WHERE AirTime > 100
LIMIT 5
""").show()

In [ ]:
Row(Year='2008', Month='1', DayofMonth='3', DayOfWeek='4', DepTime='2003', 
    CRSDepTime='1955', ArrTime='2211', CRSArrTime='2225', UniqueCarrier='WN',
    FlightNum='335', TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150',
    AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', 
    Distance='810', TaxiIn='4', TaxiOut='8', Cancelled='0', CancellationCode=None,
    Diverted='0', CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', 
    LateAircraftDelay='NA')

In [24]:
# Subqueries
df = spark.sql("""

Select Month, DayOfMonth
FROM flights
WHERE Distance =

(SELECT MAX(Distance)
FROM flights 
)LIMIT 5

""").show()

+-----+----------+
|Month|DayOfMonth|
+-----+----------+
|    1|         1|
|    1|         2|
|    1|         3|
|    1|         4|
|    1|         5|
+-----+----------+



In [3]:
# Correlated subqueries
df = spark.sql("""

select A.*,B.*
from Table_train_clicks B
right join Table_promoted_content A
on B.ad_id = A.ad_id

WHERE advertiser_id =

(SELECT MAX(advertiser_id)
FROM Table_promoted_content AS C
WHERE A.ad_id = C.ad_id
)LIMIT 5

""").show()

+-----+-----------+-----------+-------------+----------+-----+-------+
|ad_id|document_id|campaign_id|advertiser_id|display_id|ad_id|clicked|
+-----+-----------+-----------+-------------+----------+-----+-------+
|    1|       6614|          1|            7|      null| null|   null|
|    2|     471467|          2|            7|      null| null|   null|
|    3|       7692|          3|            7|      null| null|   null|
|    4|     471471|          2|            7|      null| null|   null|
|    5|     471472|          2|            7|      null| null|   null|
+-----+-----------+-----------+-------------+----------+-----+-------+



#### 1.7 Spark Sql Advanced 1 queries 

In [22]:
# Stored procedure query

df = spark.sql("""

select ad_id,
SUM(clicked) AS Totalclicked,
AVG(CAST(clicked AS DECIMAL)) AS AvgClicked
FROM Table_train_clicks
GROUP BY ad_id
ORDER BY ad_id DESC

""").show()

+-----+------------+----------+
|ad_id|Totalclicked|AvgClicked|
+-----+------------+----------+
|99876|         0.0|    0.0000|
|99819|         0.0|    0.0000|
|99803|         0.0|    0.0000|
|99800|         0.0|    0.0000|
| 9980|         0.0|    0.0000|
|99770|         0.0|    0.0000|
|99763|         0.0|    0.0000|
|99712|         0.0|    0.0000|
|99690|         1.0|    0.3333|
|99568|         4.0|    0.0625|
|99559|         0.0|    0.0000|
|99558|         0.0|    0.0000|
|99553|         0.0|    0.0000|
|99542|        17.0|    0.1954|
|99515|        17.0|    0.4359|
|99512|        40.0|    0.3738|
|99495|         4.0|    0.1905|
|99395|         0.0|    0.0000|
|99388|         2.0|    0.3333|
|99374|         0.0|    0.0000|
+-----+------------+----------+
only showing top 20 rows



In [ ]:
# Execute this query in SQL Server Developer.
# Stored procedure 

df = spark.sql("""

CREATE PROC spClicks
AS
BEGIN
SELECT ad_id,
SUM(clicked) AS Totalclicked,
AVG(CAST(clicked AS DECIMAL)) AS AvgClicked
FROM Table_train_clicks
GROUP BY ad_id
ORDER BY ad_id DESC
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute stored procedure

df = spark.sql("""

EXECUTE spClicks

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Alter Stored procedure 

df = spark.sql("""

ALTER PROC spClicks
AS
BEGIN
select ad_id,
SUM(clicked) AS Totalclicked,
AVG(CAST(clicked AS DECIMAL)) AS AvgClicked
FROM Table_train_clicks
GROUP BY ad_id
ORDER BY ad_id ASC
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Delete Stored procedure  

df = spark.sql("""

DROP PROC spClicks

""").show()

In [7]:
# Parameter query

df = spark.sql("""

select FlightNum, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
GROUP BY FlightNum
ORDER BY TotalAirTime ASC

""").show()

+---------+------------+----------+
|FlightNum|TotalAirTime|AvgAirTime|
+---------+------------+----------+
|     9743|        null|      null|
|     6917|        null|      null|
|     8911|        null|      null|
|     8910|        null|      null|
|     8941|        null|      null|
|     8940|        null|      null|
|     9742|        null|      null|
|     9002|        19.0|   19.0000|
|     7447|        35.0|   35.0000|
|     9202|        39.0|   39.0000|
|     6918|        45.0|   45.0000|
|     6902|        46.0|   46.0000|
|     6915|        46.0|   46.0000|
|     6272|        46.0|   46.0000|
|     6894|        47.0|   47.0000|
|     7720|        53.0|   53.0000|
|     6870|        60.0|   60.0000|
|     5913|        60.0|   60.0000|
|     7695|        63.0|   63.0000|
|     7476|        71.0|   35.5000|
+---------+------------+----------+
only showing top 20 rows



In [ ]:
# Execute this query in SQL Server Developer.
# Parameter 

df = spark.sql("""

CREATE PROC spFlightcriteria(@minAirTime AS INT)
AS
BEGIN
select FlightNum, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
WHERE TotalAirTime > @minAirTime
GROUP BY FlightNum
ORDER BY TotalAirTime ASC
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Executing SP parameter

EXEC spFlightcriteria 300

In [ ]:
# Execute this query in SQL Server Developer.
# Multiple INT parameters 

df = spark.sql("""

CREATE PROC spFlightcriteria(@minAirTime AS INT,
                             @maxAirTime AS INT)
AS
BEGIN
SELECT FlightNum, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
WHERE TotalAirTime >= @minAirTime AND
      TotalAirTime <= @maxAirTime AND
GROUP BY FlightNum
ORDER BY TotalAirTime ASC
END

""").show() 

In [ ]:
# Execute this query in SQL Server Developer.
# Executing SP multiple INT parameters

EXEC spFlightcriteria @minAirTime=300,@maxAirTime=500

In [49]:
# Multiple INT+VARCHAR parameters query

df = spark.sql("""

select Origin, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
GROUP BY Origin
ORDER BY TotalAirTime ASC

""").show()

+------+------------+----------+
|Origin|TotalAirTime|AvgAirTime|
+------+------------+----------+
|   PUB|        40.0|   20.0000|
|   TUP|       429.0|   42.9000|
|   PIR|       501.0|  100.2000|
|   GST|      1268.0|   15.2771|
|   BJI|      2701.0|   37.0000|
|   INL|      3233.0|   45.5352|
|   AKN|      4180.0|   37.3214|
|   SUX|      4986.0|   44.9189|
|   HTS|      5382.0|   34.7226|
|   DLG|      5626.0|   50.6847|
|   BPT|      5692.0|   22.1479|
|   ITH|      7356.0|   61.3000|
|   RHI|      8557.0|   41.3382|
|   MKG|      8630.0|   22.1282|
|   ALO|      9748.0|   36.3731|
|   PLN|     10832.0|   43.3280|
|   WRG|     11468.0|   16.7416|
|   LWB|     11553.0|   63.1311|
|   BLI|     11983.0|  100.6975|
|   ACY|     12191.0|  108.8482|
+------+------------+----------+
only showing top 20 rows



In [ ]:
# Execute this query in SQL Server Developer.
# Multiple INT+VARCHAR parameters 

df = spark.sql("""

CREATE PROC spFlightcriteria(@minAirTime AS INT,
                             @maxAirTime AS INT,
                             @Origin AS VARCHAR(MAX))
AS
BEGIN
select Origin, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
WHERE TotalAirTime >= @minAirTime AND
      TotalAirTime <= maxAirTime AND
      Origin LIKE '%' + @Origin + '%'
GROUP BY Origin
ORDER BY TotalAirTime ASC
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Executing SP multiple INT+VARCHAR parameters 

EXEC spFlightcriteria @minAirTime=300,@maxAirTime=500,@Origin='AC'

In [ ]:
# Execute this query in SQL Server Developer.
# Optional parameters

df = spark.sql("""

CREATE PROC spFlightcriteria(@minAirTime AS INT = NULL,
                             @maxAirTime AS INT = NULL,
                             @Origin AS VARCHAR(MAX))
AS
BEGIN
select Origin, 
SUM(AirTime) AS TotalAirTime,
AVG(CAST(AirTime AS DECIMAL)) AS AvgAirTime
FROM flights
WHERE (@minAirTime = NULL OR TotalAirTime >= @minAirTime) AND
      (@maxAirTime = NULL OR TotalAirTime <= @maxAirTime) AND
      Origin LIKE '%' + @Origin + '%'
GROUP BY Origin
ORDER BY TotalAirTime ASC
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Executing SP optional parameters 

EXEC spFlightcriteria @Origin='AC'

In [32]:
# Variable query. Joining 3 tables to quick view the headings. 
df = spark.sql("""

select TPC.*,TTC.*,TE.*
from Table_train_clicks TTC
inner join Table_promoted_content TPC
on TTC.ad_id = TPC.ad_id
inner join Table_events TE
on TTC.display_id = TE.display_id             

              """).show(1)

+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
|ad_id|document_id|campaign_id|advertiser_id|display_id|ad_id|clicked|display_id|          uuid|document_id|timestamp|platform|geo_location|
+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
|   35|      29658|         24|           19|      8157|   35|      0|      8157|c260dbb014dccc|    1735810|   563026|       3|   US>OH>535|
|  216|      55604|         48|           39|      9646|  216|      1|      9646|d3c5f43efb3e64|    1089240|   666570|       3|   US>AL>630|
|  304|      56139|         53|           86|     11384|  304|      0|     11384|610c151b8a2add|    1383758|   790791|       3|       AU>02|
+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
only showing 

In [37]:
# Variable query
df = spark.sql("""
select clicked 
FROM Table_train_clicks 
WHERE ad_id >= 120000

UNION ALL

select advertiser_id 
FROM Table_promoted_content 
WHERE ad_id >= 120000

UNION ALL

select platform 
FROM Table_events 
WHERE timestamp >= 800000
""").show()

+-------+
|clicked|
+-------+
|      0|
|      1|
|      0|
|      0|
|      0|
|      0|
|      0|
|      0|
|      1|
|      0|
|      0|
|      1|
|      0|
|      0|
|      1|
|      0|
|      0|
|      0|
|      0|
|      0|
+-------+
only showing top 20 rows



In [ ]:
# Execute this query in SQL Server Developer.
# Declaring/setting variables

df = spark.sql("""

DECLARE @Time AS INT
DECLARE @AD AS INT

SET @Time = 800000
SET @AD = 120000

select clicked 
FROM Table_train_clicks 
WHERE ad_id >= @AD

UNION ALL

select advertiser_id 
FROM Table_promoted_content 
WHERE ad_id >= @AD

UNION ALL

select platform 
FROM Table_events 
WHERE timestamp >= @Time

""").show()

In [51]:
# Variable query 2

df = spark.sql("""

DECLARE @Time AS INT
DECLARE @NumClicks AS INT
DECLARE @NumPromotedContent AS INT
DECLARE @NumEvents AS INT

SET @Time = >= 100000
SET @NumClicks = 12000
SET @Events = 12000
SET @NumEvents = 800000

select 'Number of clicks', @NumClicks

UNION ALL

select 'NumPromotedContent', @NumPromotedContent

UNION ALL

select 'NumEvents', @NumEvents

""").show()

+--------------------+----------------+
|    Number of clicks|scalarsubquery()|
+--------------------+----------------+
|    Number of clicks|           98162|
|Number of promote...|           88082|
|    Number of Events|           88478|
+--------------------+----------------+



In [ ]:
# Execute this query in SQL Server Developer.
# Output parameters
df = spark.sql("""

CREATE PROC spOriginInYear  (@Year AS INT,
                             @Month AS INT,
                             @OriginList AS VARCHAR(MAX) OUTPUT,
AS                           @OriginCount AS INT OUTPUT)
BEGIN

DECLARE @Origin VARCHAR(MAX)
SET @Origin = ''

SELECT @Origin = @Origin + Origin + ', '
FROM flights
WHERE Year = @Year AND Month = @Month

ORDER BY Origin ASC

SET @OriginCount = @@ROWCOUNT 
SET @OriginList = @Origin

END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute the SP OPRV

df = spark.sql("""

DECLARE @Names VARCHAR(MAX)
DECLARE @COUNT INT

EXEC spFilmsInYear
@Year = 2008,
@Month = >= 6
@OriginList = @Names OUTPUT,
@OriginCount = @Count OUTPUT

SELECT @Count AS [Number of Origin], @Names AS [List of Origin]

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Return values
df = spark.sql("""

ALTER PROC spFlightcriteria(@Year AS INT,
                            @Month AS INT)
AS
BEGIN

SELECT Origin 
FROM flights
WHERE Year = @Year AND Month = @Month

ORDER BY Origin ASC

RETURN @@ROWCOUNT

END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute SP return values

df = spark.sql("""

DECLARE @Count INT

EXEC @COUNT = spOriginInYear @Year = 2008 AND @Month = > 6

SELECT @COUNT AS [Number of Origin]

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Stored procedure IF statement 1 

df = spark.sql("""

DECLARE @IND INT
DECLARE @JAX INT

SET @IND INT = (SELECT COUNT(*) FROM flights WHERE Origin = IND)
SET @JAX INT = (SELECT COUNT(*) FROM flights WHERE Origin = JAX)

IF @IND < 8 
BEGIN
PRINT 'Attention needed'
PRINT 'There are too less IND flights'
END
ELSE
BEGIN
PRINT 'Attention needed'
PRINT 'There are too many IND flights'
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Stored procedure Nested IF statements 

df = spark.sql("""

DECLARE @IND INT
DECLARE @JAX INT

SET @IND INT = (SELECT COUNT(*) FROM flights WHERE Origin = IND)
SET @JAX INT = (SELECT COUNT(*) FROM flights WHERE Origin = JAX)

IF @IND < 8 
BEGIN
PRINT 'Attention needed'
PRINT 'There are too less IND flights'
IF @JAX > 12
BEGIN
PRINT 'However, The high number of JAX flights compensate enough for the low number of IND flights'
END
ELSE
BEGIN
PRINT 'And The low number of JAX flights do not compensate enough for the low number of IND flights'
END
ELSE
BEGIN
PRINT 'Attention needed'
PRINT 'There are too many IND flights'
END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Stored procedure IF statement 2

df = spark.sql("""

CREATE PROC spVariableData
(

@InfoType VARCHAR(9)

)
AS
BEGIN

        IF @InfoType='ALL'
        BEGIN
       (SELECT * FROM flights)
        RETURN
        END
        
        IF @InfoType='DATE'
        BEGIN
        (SELECT Year, Month, DayofMonth, DayOfWeek FROM flights)
        RETURN
        END
        
        IF @InfoType='TIME'
        BEGIN
        (SELECT DepTime, CRSDepTime, ArrTime, CRSArrTime, ActualElapsedTime, CRSElapsedTime, AirTime)
        RETURN
        END
        
        IF @InfoType='FLIGHTINFO'
        BEGIN
        (SELECT UniqueCarrier, FlightNum, TailNum, Origin, Dest, Distance)
        RETURN
        END
        
        SELECT 'Please choose 'ALL','DATE','TIME' or 'FLIGHTINFO'

END


""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute SP IF statement 2 

EXEC spVariableData @InfoType = 'ALL'
EXEC spVariableData @InfoType = 'DATE'
EXEC spVariableData @InfoType = 'TIME'
EXEC spVariableData @InfoType = 'FLIGHTINFO'

In [ ]:
# Execute this query in SQL Server Developer.
# WHILE LOOP 1

df = spark.sql("""

DECLARE @Counter INT

SET @COUNTER = 1

WHILE @COUNTER<=10
    BEGIN
        PRINT @COUNTER
        SET @COUNTER = @COUNTER + 1
    END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# WHILE LOOP 2

df = spark.sql("""

DECLARE @Counter INT
DECLARE @MaxAirtime INT
DECLARE @NumOrigin INT

SET @MaxAirtime = (SELECT MAX(AirTime) FROM flights
SET @COUNTER = 0

WHILE @COUNTER<=@MaxAirtime
    BEGIN
        SET @NumOrigin=
        (SELECT COUNT(*) FROM flights WHERE AirTime = @Counter)
        
        IF @NumOrigin=0 BREAK
        
        PRINT CAST(@NumOrigin AS VARCHAR(3)) + 'Origin has flew'
            CAST(@COUNTER AS VARCHAR(3)) + '  In AirTime'
    END

""").show()

In [ ]:
# Execute this query in SQL Server Developer.
# WHILE LOOP 3

df = spark.sql("""



""").show()

In [26]:
# WHILE LOOP 3. Joining 3 the tables and quick-viewing the headings. 
df = spark.sql("""

select TPC.*,TTC.*,TE.*
from Table_train_clicks TTC
inner join Table_promoted_content TPC
on TTC.ad_id = TPC.ad_id
inner join Table_events TE
on TTC.display_id = TE.display_id             

              """).show(1)

+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
|ad_id|document_id|campaign_id|advertiser_id|display_id|ad_id|clicked|display_id|          uuid|document_id|timestamp|platform|geo_location|
+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
|   35|      29658|         24|           19|      8157|   35|      0|      8157|c260dbb014dccc|    1735810|   563026|       3|   US>OH>535|
+-----+-----------+-----------+-------------+----------+-----+-------+----------+--------------+-----------+---------+--------+------------+
only showing top 1 row



In [ ]:
# Execute this query in SQL Server Developer.
# WHILE LOOP 3.
df = spark.sql("""

DECLARE @Adid INT 
DECLARE @Platform VARCHAR(MAX)

DECLARE AdCursor CURSOR FOR
SELECT (SELECT ad_id FROM Table_train_clicks),
SELECT (SELECT ad_id FROM Table_events)

OPEN AdCursor

FETCH NEXT FROM AdCursor INTO @Adid,@Platform

WHILE @@FETCH_STATUS = 0
    BEGIN
        PRINT 'Type of platfom' + @Platform
        SELECT Castgeo_location FROM Table_events WHERE Castad_id = @ADid
        
        FETCH NEXT FROM AdCursor INTO @ADid, @Platform
    END
    
CLOSE AdCursor
DEALLOCATE AdCursor



              """).show(1)

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 1
df = spark.sql("""

EXEC ('SELECT * FROM Table_train_clicks')

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 2
df = spark.sql("""

EXEC sp_executesql N'SELECT * FROM Table_train_clicks)

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 3
df = spark.sql("""

DECLARE @Tablename NVARCHAR(128)
DECLARE @SQLString NVARCHAR(MAX)

SET @Tablename = N'Table_train_clicks'

SET @SQLString = N'SELECT * FROM' + @Tablename

EXEC sp_executesql @SQLString

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 4
df = spark.sql("""

DECLARE @Number INT
DECLARE @NumberString NVARCHAR(4)
DECLARE @SQLString NVARCHAR(MAX)

SET @Number = 10

SET @NumberString = CAST(@Number AS NVARCHAR(4))

SET @SQLString = N'SELECT TOP ' + @NumberString + ' * FROM Table_train_clicks ORDER BY ad_id'

EXEC sp_executesql @SQLString

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 5 - SP
df = spark.sql("""

CREATE PROC spVariableTable
(

        @TableName NVARCHAR(128)
)
AS
BEGIN

DECLARE @SQLString NVARCHAR(MAX)

SET  @SQLString = N'SELECT * FROM ' + @TableName

EXEC sp_executesql @SQLString

END

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute dynamic Stored procedure

spVariableTable 'Table_train_clicks'

In [ ]:
# Execute this query in SQL Server Developer.
# Dynamic SQL 5 - SP multiple parameters
df = spark.sql("""

ALTER PROC spVariableTable
(

        @TableName NVARCHAR(128),
        @Number INT
)
AS
BEGIN

DECLARE @SQLString NVARCHAR(MAX)
DECLARE @NumberString NVARCHAR(4)

SET @NumberString = CAST(@Number AS NVARCHAR(4))

SET  @SQLString = N'SELECT TOP ' + @NumberString + ' * FROM ' + @TableName

EXEC sp_executesql @SQLString

END

              """).show()

In [ ]:
# Execute this query in SQL Server Developer.
# Execute dynamic Stored procedure multiple parameters

spVariableTable 'Table_train_clicks', 10